In [8]:
import os
import pandas as pd
from owlready2 import get_ontology, Thing, DataProperty, ObjectProperty, Imp, FunctionalProperty, AllDisjoint

# 1. Base Path 설정
base_dir = "/home/min/main/Ph2"
ontology_path = os.path.join(base_dir, "PH2_tbox_only.owl")
images_base_dir = os.path.join(base_dir, "PH2 Dataset images")

# 2. TBox 구성: 온톨로지 구조 정의 (이미지 관련 클래스 포함)
onto = get_ontology("http://min.org/PH2_ontology.owl")

with onto:
    # SkinLesion 계층
    class SkinLesion(Thing):
        """A general skin lesion from PH2 dataset"""
        pass

    class BenignLesion(SkinLesion):
        pass

    class MalignantLesion(SkinLesion):
        pass

    AllDisjoint([BenignLesion, MalignantLesion])

    # Diagnosis 계층
    class Diagnosis(Thing):
        """Histological Diagnosis concept."""
        pass

    class BenignDiagnosis(Diagnosis):
        pass

    class MalignantDiagnosis(Diagnosis):
        pass

    AllDisjoint([BenignDiagnosis, MalignantDiagnosis])

    class BlueNevusDiagnosis(BenignDiagnosis):
        pass

    class DysplasticNevusDiagnosis(BenignDiagnosis):
        pass

    class IntradermalNevusDiagnosis(BenignDiagnosis):
        pass

    class MelanomaDiagnosis(MalignantDiagnosis):
        pass

    class NodularMelanomaDiagnosis(MalignantDiagnosis):
        pass

    class LentigoMalignaDiagnosis(MalignantDiagnosis):
        pass

    class MissingDiagnosis(Diagnosis):
        """Case where diagnosis was missing/NaN"""
        pass

    # 객체 속성: SkinLesion와 Diagnosis 연결
    class hasDiagnosis(ObjectProperty):
        domain = [SkinLesion]
        range = [Diagnosis]

    # 데이터 속성: 임상적 특성
    class hasAsymmetry(DataProperty, FunctionalProperty):
        domain = [SkinLesion]
        range = [int]

    class hasColor_White(DataProperty, FunctionalProperty):
        domain = [SkinLesion]
        range = [bool]

    class hasColor_Red(DataProperty, FunctionalProperty):
        domain = [SkinLesion]
        range = [bool]

    class hasColor_LightBrown(DataProperty, FunctionalProperty):
        domain = [SkinLesion]
        range = [bool]

    class hasColor_DarkBrown(DataProperty, FunctionalProperty):
        domain = [SkinLesion]
        range = [bool]

    class hasColor_BlueGray(DataProperty, FunctionalProperty):
        domain = [SkinLesion]
        range = [bool]

    class hasColor_Black(DataProperty, FunctionalProperty):
        domain = [SkinLesion]
        range = [bool]

    class hasPigmentNetwork(DataProperty, FunctionalProperty):
        domain = [SkinLesion]
        range = [str]

    class hasDotsGlobules(DataProperty, FunctionalProperty):
        domain = [SkinLesion]
        range = [str]

    class hasStreaks(DataProperty, FunctionalProperty):
        domain = [SkinLesion]
        range = [str]

    class hasRegressionAreas(DataProperty, FunctionalProperty):
        domain = [SkinLesion]
        range = [str]

    class hasBlueWhitishVeil(DataProperty, FunctionalProperty):
        domain = [SkinLesion]
        range = [str]

    # 동등성(Equivalence) 정의: 진단에 따라 Lesion을 자동 분류
    MalignantLesion.equivalent_to.append(
        SkinLesion & (hasDiagnosis.some(MalignantDiagnosis))
    )
    BenignLesion.equivalent_to.append(
        SkinLesion & (hasDiagnosis.some(BenignDiagnosis))
    )

    # SWRL 규칙: HighAsymmetryLesion, VerySuspiciousLesion
    class HighAsymmetryLesion(SkinLesion):
        """Lesion with Asymmetry >= 2"""
        pass

    rule_high_asym = Imp("rule_high_asymmetry", namespace=onto)
    rule_high_asym.rule = """
        SkinLesion(?x) ^ hasAsymmetry(?x, ?a) ^
        swrlb:greaterThanOrEqual(?a, 2)
        -> HighAsymmetryLesion(?x)
    """
    rule_high_asym.is_rule = True

    class VerySuspiciousLesion(SkinLesion):
        """Lesion that is very suspicious based on color + high asymmetry."""
        pass

    rule_suspicious = Imp("rule_very_suspicious", namespace=onto)
    rule_suspicious.rule = """
        SkinLesion(?x) ^ hasAsymmetry(?x, ?a) ^
        swrlb:greaterThanOrEqual(?a, 2) ^
        hasColor_Black(?x, ?b) ^ swrlb:booleanEqual(?b, true)
        -> VerySuspiciousLesion(?x)
    """
    rule_suspicious.is_rule = True

    # 추가 SWRL 규칙 1: SuspiciousColorLesion  
    # (어두운 갈색과 청회색이 모두 true인 경우, 색상 패턴상 의심스러운 병변으로 분류)
    class SuspiciousColorLesion(SkinLesion):
        """Lesion with a suspicious color pattern (both dark brown and blue-gray present)."""
        pass

    rule_suspicious_color = Imp("rule_suspicious_color", namespace=onto)
    rule_suspicious_color.rule = """
        SkinLesion(?x) ^ hasColor_DarkBrown(?x, ?d) ^ hasColor_BlueGray(?x, ?c) ^
        swrlb:booleanEqual(?d, true) ^ swrlb:booleanEqual(?c, true)
        -> SuspiciousColorLesion(?x)
    """
    rule_suspicious_color.is_rule = True

    # 추가 SWRL 규칙 2: PossibleDysplasticNevus  
    # (비대칭성은 상대적으로 낮고, PigmentNetwork와 DotsGlobules가 atypical (AT)인 경우)
    class PossibleDysplasticNevus(SkinLesion):
        """Lesion that might be dysplastic nevus based on atypical pigment network and dots/globules."""
        pass

    rule_dysplastic = Imp("rule_dysplastic_nevus", namespace=onto)
    rule_dysplastic.rule = """
        SkinLesion(?x) ^ hasAsymmetry(?x, ?a) ^ swrlb:lessThan(?a, 2) ^
        hasPigmentNetwork(?x, ?pn) ^ swrlb:stringEqual(?pn, "AT") ^
        hasDotsGlobules(?x, ?dg) ^ swrlb:stringEqual(?dg, "AT")
        -> PossibleDysplasticNevus(?x)
    """
    rule_dysplastic.is_rule = True

    # 이미지 데이터 관련 클래스 정의
    class ImageData(Thing):
        """Represents an image file associated with a lesion."""
        pass

    class hasFilePath(DataProperty, FunctionalProperty):
        domain = [ImageData]
        range = [str]

    class hasTitle(DataProperty, FunctionalProperty):
        domain = [ImageData]
        range = [str]

    # SkinLesion와 ImageData를 연결하는 객체 속성
    class hasImage(ObjectProperty):
        domain = [SkinLesion]
        range = [ImageData]

# TBox 저장
onto.save(os.path.join(base_dir, "PH2_tbox_only.owl"), format="rdfxml")
print("[INFO] TBox ontology structure saved as PH2_tbox_only.owl")

# 3. CSV 데이터를 이용해 ABox 생성 (메타데이터와 이미지 정보 통합)
df = pd.read_csv(os.path.join(base_dir, "Ph2_dataset_2.csv"), header=1)
df.rename(columns={
    "Asymmetry\n(0/1/2)": "Asymmetry",
    "Pigment Network\n(AT/T)": "PigmentNetwork",
    "Dots/Globules\n(A/AT/T)": "DotsGlobules_A",
    "Streaks\n(A/P)": "Streaks",
    "Regression Areas\n(A/P)": "RegressionAreas",
    "Blue-Whitish Veil\n(A/P)": "BlueWhitishVeil"
}, inplace=True)

# TBox를 저장한 파일로부터 온톨로지 로드
onto = get_ontology(os.path.join(base_dir, "PH2_tbox_only.owl")).load()

with onto:
    # 기존 클래스 및 속성 가져오기
    SkinLesion = onto.SkinLesion

    BlueNevusDiagnosis = onto.BlueNevusDiagnosis
    DysplasticNevusDiagnosis = onto.DysplasticNevusDiagnosis
    IntradermalNevusDiagnosis = onto.IntradermalNevusDiagnosis
    MelanomaDiagnosis = onto.MelanomaDiagnosis
    NodularMelanomaDiagnosis = onto.NodularMelanomaDiagnosis
    LentigoMalignaDiagnosis = onto.LentigoMalignaDiagnosis
    MissingDiagnosis = onto.MissingDiagnosis

    hasAsymmetry = onto.hasAsymmetry
    hasColor_White = onto.hasColor_White
    hasColor_Red = onto.hasColor_Red
    hasColor_LightBrown = onto.hasColor_LightBrown
    hasColor_DarkBrown = onto.hasColor_DarkBrown
    hasColor_BlueGray = onto.hasColor_BlueGray
    hasColor_Black = onto.hasColor_Black
    hasPigmentNetwork = onto.hasPigmentNetwork
    hasDotsGlobules = onto.hasDotsGlobules
    hasStreaks = onto.hasStreaks
    hasRegressionAreas = onto.hasRegressionAreas
    hasBlueWhitishVeil = onto.hasBlueWhitishVeil
    hasDiagnosis = onto.hasDiagnosis

    ImageData = onto.ImageData
    hasFilePath = onto.hasFilePath
    hasTitle = onto.hasTitle
    hasImage = onto.hasImage

    diagnosis_map = {
        "Histological Diagnosis_Blue Nevus": onto.BlueNevusDiagnosis,
        "Histological Diagnosis_Dysplastic Nevus": onto.DysplasticNevusDiagnosis,
        "Histological Diagnosis_Intradermal Nevus": onto.IntradermalNevusDiagnosis,
        "Histological Diagnosis_Melanoma": onto.MelanomaDiagnosis,
        "Histological Diagnosis_Nodular Melanoma": onto.NodularMelanomaDiagnosis,
        "Histological Diagnosis_Lentigo Maligna": onto.LentigoMalignaDiagnosis,
        "Histological Diagnosis_Missing": onto.MissingDiagnosis
    }

    # 각 CSV 행을 순회하며 인스턴스 생성
    for idx, row in df.iterrows():
        case_id = row["Image Name"]
        lesion_name = f"Lesion_{case_id}"
        lesion_inst = SkinLesion(lesion_name)

        # 임상적 속성 할당
        lesion_inst.hasAsymmetry = int(row["Asymmetry"])
        lesion_inst.hasColor_White = bool(row["White"])
        lesion_inst.hasColor_Red = bool(row["Red"])
        lesion_inst.hasColor_LightBrown = bool(row["Light-Brown"])
        lesion_inst.hasColor_DarkBrown = bool(row["Dark-Brown"])
        lesion_inst.hasColor_BlueGray = bool(row["Blue-Gray"])
        lesion_inst.hasColor_Black = bool(row["Black"])

        pn_val = row["PigmentNetwork"]
        if pd.isna(pn_val):
            pn_val = "none"
        lesion_inst.hasPigmentNetwork = pn_val

        dots_val = row["DotsGlobules_A"]
        if pd.isna(dots_val):
            dots_val = "none"
        lesion_inst.hasDotsGlobules = dots_val

        st_val = row["Streaks"]
        if pd.isna(st_val):
            st_val = "none"
        lesion_inst.hasStreaks = st_val

        reg_val = row["RegressionAreas"]
        if pd.isna(reg_val):
            reg_val = "none"
        lesion_inst.hasRegressionAreas = reg_val

        bw_val = row["BlueWhitishVeil"]
        if pd.isna(bw_val):
            bw_val = "none"
        lesion_inst.hasBlueWhitishVeil = bw_val

        # 진단 할당: CSV의 해당 진단 컬럼 중 True인 경우 할당, 없으면 MissingDiagnosis
        found_dx = False
        for col_dia, dx_class in diagnosis_map.items():
            if col_dia in df.columns:
                if row[col_dia] == True:
                    dx_inst = dx_class(f"{dx_class.__name__}_{case_id}")
                    lesion_inst.hasDiagnosis.append(dx_inst)
                    found_dx = True
                    break
        if not found_dx:
            dx_inst = MissingDiagnosis(f"MissingDiagnosis_{case_id}")
            lesion_inst.hasDiagnosis.append(dx_inst)

        # 이미지 인스턴스 생성 및 연결
        image_path = os.path.join(images_base_dir, case_id, f"{case_id}_Dermoscopic_Image", f"{case_id}.bmp")
        image_inst = ImageData(f"Image_{case_id}")
        image_inst.hasFilePath = image_path
        image_inst.hasTitle = case_id
        lesion_inst.hasImage.append(image_inst)

# ABox 저장
onto.save(os.path.join(base_dir, "PH2_ontology_with_individuals.owl"), format="rdfxml")
print("[INFO] Full ontology (TBox + ABox) saved as PH2_ontology_with_individuals.owl")


[INFO] TBox ontology structure saved as PH2_tbox_only.owl
[INFO] Full ontology (TBox + ABox) saved as PH2_ontology_with_individuals.owl


In [9]:
from owlready2 import sync_reasoner_pellet
sync_reasoner_pellet(debug=True)

for lesion in onto.SkinLesion.instances():
    print(lesion.name, lesion.is_a)

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlready2/pellet/log4j-1.2-api-2.19.0.jar:/home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlready2/pellet/xml-apis-1.4.01.jar:/home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlready2/pellet/jena-arq-2.10.0.jar:/home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlready2/pellet/antlr-runtime-3.2.jar:/home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlready2/pellet/httpclient-4.2.3.jar:/home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlready2/pellet/jgrapht-jdk1.5.jar:/home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlready2/pellet/jena-iri-0.9.5.jar:/home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlread

Lesion_IMD003 [PH2_ontology_with_individuals.SkinLesion, PH2_ontology_with_individuals.SkinLesion]
Lesion_IMD009 [PH2_ontology_with_individuals.SkinLesion, PH2_ontology_with_individuals.SkinLesion]
Lesion_IMD016 [PH2_ontology_with_individuals.SkinLesion, PH2_ontology_with_individuals.SkinLesion]
Lesion_IMD022 [PH2_ontology_with_individuals.SkinLesion, PH2_ontology_with_individuals.SkinLesion]
Lesion_IMD024 [PH2_ontology_with_individuals.SkinLesion, PH2_ontology_with_individuals.SkinLesion]
Lesion_IMD025 [PH2_ontology_with_individuals.SkinLesion, PH2_ontology_with_individuals.SkinLesion]
Lesion_IMD035 [PH2_ontology_with_individuals.SkinLesion, PH2_ontology_with_individuals.SkinLesion]
Lesion_IMD038 [PH2_ontology_with_individuals.SkinLesion, PH2_ontology_with_individuals.SkinLesion]
Lesion_IMD042 [PH2_ontology_with_individuals.SkinLesion, PH2_ontology_with_individuals.SkinLesion]
Lesion_IMD044 [PH2_ontology_with_individuals.SkinLesion, PH2_ontology_with_individuals.SkinLesion]
Lesion_IMD

* Owlready2 * Pellet took 0.8289628028869629 seconds
* Owlready * Reparenting  -> : {swrl.Imp} => {owl.Thing}
* Owlready * Reparenting  -> : {swrl.Imp} => {owl.Thing}
* Owlready * Reparenting  -> : {swrl.Imp, owl.Thing} => {owl.Thing}
* Owlready * Reparenting  -> : {swrl.Imp, owl.Thing} => {owl.Thing}
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [ ]:
from owlready2 import sync_reasoner_pellet

sync_reasoner_pellet(debug=True)

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlready2/pellet/log4j-1.2-api-2.19.0.jar:/home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlready2/pellet/xml-apis-1.4.01.jar:/home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlready2/pellet/jena-arq-2.10.0.jar:/home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlready2/pellet/antlr-runtime-3.2.jar:/home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlready2/pellet/httpclient-4.2.3.jar:/home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlready2/pellet/jgrapht-jdk1.5.jar:/home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlready2/pellet/jena-iri-0.9.5.jar:/home/min/anaconda3/envs/ph2/lib/python3.12/site-packages/owlread